In [ ]:
weatherApi = WorldWeatherApi()

In [ ]:
# df = weatherApi.test_download(city="sao+paulo", start_date="2025-07-14", end_date="2025-07-14", freq=1)

In [ ]:
import os
import pandas as pd

def load_weather_data(folder_path):
    # Lista para armazenar os dataframes de cada cidade
    dfs = []
    
    # Iterar sobre todos os arquivos CSV na pasta
    for filename in os.listdir(folder_path):
        print(filename)
        if filename.endswith(".csv"):
            # Criar o caminho completo do arquivo
            file_path = os.path.join(folder_path, filename)
            
            # Carregar o arquivo CSV em um dataframe
            df = pd.read_csv(file_path)
            
            # Extrair o nome do município (antes da vírgula e 'Portugal')
            city_name = filename.split(',')[0]  # Pega tudo antes da vírgula

            # Substituir "+" por espaço (" ") no nome do município
            city_name = city_name.replace("+", " ")
            
            # Adicionar o nome da cidade como uma coluna
            df['municipio'] = city_name
            
            # Adicionar o dataframe à lista
            dfs.append(df)
    
    # Concatenar todos os dataframes em um único dataframe
    combined_df = pd.concat(dfs, ignore_index=True)
    
    return combined_df


In [ ]:
import pandas as pd

def generate_city_ranking(df, aggregation='mean', temp_min=20, temp_max=25, humidity_min=40, humidity_max=60, uv_max=5, precip_max=1, wind_speed_max=20):
    """
    Função para gerar o ranking de clima agradável por cidade, calculando a média ou mediana das variáveis,
    com pesos ajustados para cada variável.
    
    Parâmetros:
    - df (DataFrame): Dados climáticos diários.
    - aggregation (str): Tipo de agregação ('mean' ou 'median').
    - temp_min (int): Temperatura mínima aceitável para ser considerada agradável (Celsius).
    - temp_max (int): Temperatura máxima aceitável para ser considerada agradável (Celsius).
    - humidity_min (int): Umidade mínima aceitável.
    - humidity_max (int): Umidade máxima aceitável.
    - uv_max (int): Índice UV máximo aceitável.
    - precip_max (float): Precipitação máxima aceitável (em mm).
    - wind_speed_max (int): Velocidade máxima do vento aceitável (em km/h).
    
    Retorna:
    - DataFrame com as pontuações de clima agradável para cada cidade.
    """
    
    # Verificar se a coluna 'municipio' está presente no DataFrame
    if 'municipio' not in df.columns:
        raise ValueError("A coluna 'municipio' não foi encontrada no DataFrame.")
    
    # Agrupar por cidade e calcular a média ou mediana
    if aggregation == 'mean':
        df_agg = df.groupby('municipio').mean()
    elif aggregation == 'median':
        df_agg = df.groupby('municipio').median()
    else:
        raise ValueError("O parâmetro 'aggregation' deve ser 'mean' ou 'median'.")
    
    # Restaurar 'municipio' como coluna após o agrupamento
    df_agg['municipio'] = df_agg.index
    
    # Verificar se as colunas essenciais estão presentes após o agrupamento
    required_columns = ['maxtempC', 'mintempC', 'humidity', 'uvIndex', 'precipMM', 'windspeedKmph', 'FeelsLikeC']
    missing_columns = [col for col in required_columns if col not in df_agg.columns]
    if missing_columns:
        raise ValueError(f"As seguintes colunas estão faltando no DataFrame após o agrupamento: {', '.join(missing_columns)}")
    
    # Função para calcular a pontuação de cada variável
    def score_temperature(temp):
        if temp < temp_min:
            return 0
        elif temp > temp_max:
            return 0
        else:
            return 10  # Pontuação máxima para temperatura dentro da faixa agradável

    def score_humidity(humidity):
        if humidity < humidity_min or humidity > humidity_max:
            return 0
        else:
            return 10  # Pontuação máxima para umidade dentro da faixa agradável

    def score_uv(uv):
        if uv <= uv_max:
            return 10
        else:
            return 0

    def score_precipitation(precip):
        if precip <= precip_max:
            return 10
        else:
            return 0

    def score_wind_speed(wind_speed):
        if wind_speed <= wind_speed_max:
            return 10
        else:
            return 0

    def score_feels_like(feels_like, temp):
        if abs(feels_like - temp) <= 2:  # Se a sensação térmica estiver muito próxima da temperatura real
            return 10
        else:
            return 0
    
    # Aplicar as funções de pontuação para cada cidade
    df_agg['temp_score'] = df_agg['maxtempC'].apply(score_temperature)
    df_agg['humidity_score'] = df_agg['humidity'].apply(score_humidity)
    df_agg['uv_score'] = df_agg['uvIndex'].apply(score_uv)
    df_agg['precip_score'] = df_agg['precipMM'].apply(score_precipitation)
    df_agg['wind_score'] = df_agg['windspeedKmph'].apply(score_wind_speed)
    df_agg['feels_like_score'] = df_agg.apply(lambda row: score_feels_like(row['FeelsLikeC'], row['maxtempC']), axis=1)
    
    # Definir os pesos para cada variável
    temp_weight = 0.6
    feels_like_weight = 0.2
    other_weight = 0.2 / 4  # O restante será dividido entre as 4 outras variáveis
    
    # Calcular a pontuação total com os pesos ajustados
    df_agg['total_score'] = (
        (df_agg['temp_score'] * temp_weight) + 
        (df_agg['feels_like_score'] * feels_like_weight) + 
        (df_agg['humidity_score'] * other_weight) + 
        (df_agg['uv_score'] * other_weight) + 
        (df_agg['precip_score'] * other_weight) + 
        (df_agg['wind_score'] * other_weight)
    )
    
    # Normalizar a pontuação para que o total máximo seja 100
    df_agg['normalized_score'] = df_agg['total_score'] * 100  # Já está em uma escala de 0 a 100
    
    # Ordenar as cidades pelo ranking
    df_sorted = df_agg.sort_values(by='normalized_score', ascending=False)
    
    # Exibir as cidades com o clima mais agradável no topo
    return df_sorted[['municipio', 'maxtempC', 'mintempC', 'humidity', 'precipMM', 'windspeedKmph','temp_score', 'feels_like_score', 'humidity_score', 'uv_score', 'precip_score', 'wind_score', 'total_score', 'normalized_score']]

# Exemplo de uso:
# df_ranking = generate_city_ranking(df, aggregation='mean')
# ace_tools.display_dataframe_to_user(name="Ranking de Clima Agradável por Cidade", dataframe=df_ranking)


In [ ]:
filePathClima = '..\\..\\Bases\\Clima\\worldWeatherApi\\'
df = load_weather_data(filePathClima)

In [ ]:
df

In [ ]:
# Exemplo de uso:
df_ranking = generate_city_ranking(df)


In [ ]:
df_ranking

In [ ]:
# export df
filePathClimaRaking = filePathClima + '\\ranking\\' + 'ClimaRaking' + '.csv'
df_ranking.to_csv(filePathClimaRaking, sep=',', encoding='utf-8', doublequote=True, decimal=',')

In [ ]:
def show_sorted_weather(df, score_type="normalized_score"):
    """
    Função para mostrar o clima de forma ordenada, classificando pela pontuação total ou normalizada.
    
    Parâmetros:
    - df (DataFrame): Dados climáticos com a coluna de pontuação.
    - score_type (str): Tipo de pontuação para ordenação. Pode ser "total_score" ou "normalized_score".
    
    Retorna:
    - DataFrame ordenado com base no tipo de pontuação escolhido.
    """
    # Verificar se a coluna de pontuação escolhida existe no DataFrame
    if score_type not in df.columns:
        raise ValueError(f"A coluna '{score_type}' não foi encontrada no DataFrame.")
    
    # Verificar se as colunas essenciais estão presentes no DataFrame
    required_columns = ['municipio', 'maxtempC', 'mintempC', 'humidity', 'precipMM', 'windspeedKmph']
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"As seguintes colunas estão faltando no DataFrame: {', '.join(missing_columns)}")
    
    # Ordenar o DataFrame com base no score escolhido (de forma decrescente)
    sorted_df = df.sort_values(by=score_type, ascending=False)
    
    # Exibir as cidades com o clima mais agradável no topo
    return sorted_df[['municipio', score_type, 'maxtempC', 'mintempC', 'humidity', 'precipMM', 'windspeedKmph']]

# Exemplo de uso:
# df_sorted = show_sorted_weather(df, score_type="normalized_score")
# ace_tools.display_dataframe_to_user(name="Ranking de Clima Agradável", dataframe=df_sorted)


In [ ]:
df

In [ ]:
show_sorted_weather(df_ranking)